In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 2.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=1201034 sha256=0457db55f3200f8e73d102bc4e83e17e7a6d5408689e5406b22e2da869eda6a4
  Stored in directory: /home/giangqd/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import cross_validate
from surprise import SVD

In [25]:
def get_pivot_data():
    triplets = pd.read_csv('data/triplets_file.csv')
    songs = triplets.groupby('song_id').sum().sort_values('listen_count', ascending=False)
    songs = songs[songs['listen_count'] < 100]

    # delete songs with less than 100 listens
    triplets = triplets[~triplets['song_id'].isin(songs.index)]

    # all listen counts greater than 130 are set to 130
    triplets['listen_count'] = triplets['listen_count'].apply(lambda x: 10 if x > 10 else x)
    
    # use the first 500000 rows
    triplets = triplets.head(500000)

    # add personal data
    me = pd.read_csv('data/me.csv')
    triplets = pd.concat([me, triplets], ignore_index=True)
    
    # get list of songs and total listen count
    songs = triplets.groupby('song_id').sum().sort_values('listen_count', ascending=False)
    return triplets.pivot(index='user_id', columns='song_id', values='listen_count').fillna(0), songs, triplets
    #return triplets.pivot(index='user_id', columns='song_id', values='listen_count'), songs

In [4]:
song_data = pd.read_csv('data/meta_data.csv')

In [26]:
pivot, songs_list, songs_triplet = get_pivot_data()

/tmp/ipykernel_5879/1360975585.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  songs = triplets.groupby('song_id').sum().sort_values('listen_count', ascending=False)
/tmp/ipykernel_5879/1360975585.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  songs = triplets.groupby('song_id').sum().sort_values('listen_count', ascending=False)


In [ ]:
df_ratings = pivot

In [10]:
df_ratings_dummy = pivot
df_ratings_dummy

song_id,SOAAAGQ12A8C1420C8,SOAACPJ12A81C21360,SOAACSG12AB018DC80,SOAAEJI12AB0188AB5,SOAAFAC12A67ADF7EB,SOAAFYH12A8C13717A,SOAAJMQ12A6D4F7D17,SOAAKPM12A58A77210,SOAALWN12A6D4F7FDA,SOAAMOW12AB018149B,...,SOZZKPR12A6D4F8147,SOZZLTY12A67AE0AD0,SOZZLZN12A8AE48D6D,SOZZRHE12A6702165F,SOZZTCU12AB0182C58,SOZZTNF12A8C139916,SOZZVWB12AB0189C30,SOZZWZV12A67AE140F,SOZZYAO12A6701FF36,SOZZZPV12A8C1444B5
user_id,,,,,,,,,,,,,,,,,,,,,
00003a4459f33b92906be11abe0e93efc423c0ff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000b474f815bcff17a4bc9ce5324f9352dafe07d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000b4e4134d5f77d7608fbf86fb3e1adac4478a8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000d2df2cb8ad7300f89512f8fe8fadc4f99e733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff1d613baa7ce8e5acb7181093d9683804a55b6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff300cd094fe04030b79fae550dc9d065190182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff543db7918cb8f4f56f7470903eb2f1d5a6dd8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
similarity_matrix = cosine_similarity(df_ratings_dummy, df_ratings_dummy)

In [12]:
similarity_matrix_df = pd.DataFrame(similarity_matrix, index=df_ratings.index, columns=df_ratings.index)

In [22]:
def calculate_ratings(id_movie, id_user):
    if id_movie in df_ratings.columns:
        cosine_scores = similarity_matrix_df[id_user] #similarity of id_user with every other user
        print(cosine_scores)
        ratings_scores = df_ratings[id_movie]      #ratings of every other user for the movie id_movie
        print(ratings_scores)
        index_not_rated = ratings_scores[ratings_scores.isnull()].index
        ratings_scores = ratings_scores.dropna()
        cosine_scores = cosine_scores.drop(index_not_rated)
        ratings_movie = np.dot(ratings_scores, cosine_scores)/cosine_scores.sum()
    else:
        return 2.5
    return ratings_movie

In [28]:


reader = Reader()
data = Dataset.load_from_df(songs_triplet, reader)
knn = KNNBasic()
cross_validate(knn, data, measures=['RMSE', 'mae'], cv = 3)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([2.44757379, 2.4574696 , 2.46852101]),
 'test_mae': array([1.61908483, 1.62710689, 1.62950261]),
 'fit_time': (12.770323038101196, 13.74442172050476, 13.463751792907715),
 'test_time': (21.022218227386475, 21.214932680130005, 21.602307319641113)}

In [29]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'mae'], cv = 3)

{'test_rmse': array([2.26292261, 2.26284807, 2.27020975]),
 'test_mae': array([1.56544944, 1.56358034, 1.5697174 ]),
 'fit_time': (5.304940223693848, 5.74248743057251, 5.722255706787109),
 'test_time': (2.016486883163452, 2.0905871391296387, 2.1675543785095215)}

In [33]:
svd.predict('1234_thisisme','SOKLRPJ12A8C13C3FE').est

4.041425392307486

In [32]:
svd.predict('1234_thisisme','SOFPZCI12A58A7CF79').est

2.117356047874897